In [1]:
import pandas as pd 
import numpy as np
import random
import os 
import pymysql
import json
import pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle 
from sklearn.model_selection import train_test_split

In [2]:
default_path = '../'
data_path = os.path.join(default_path, 'data')

In [3]:
dsm_data = pd.read_csv(os.path.join(data_path, 'dsm_data.csv'))
dsm_data

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,text,label
0,al526,"so, am i depressed? i'm honestly unsure myself...",0
1,as8qg,"i originally put it in askreddit, but realized...",0
2,as8qg,how did you do it? am i even depressed at all?...,0
3,ay63c,have been depressed for approximately 20 years...,0
4,aycre,i don't know if i'm depressed or if i'm being ...,0
...,...,...,...
301147,126547,It is said that the incidence of respiratory d...,9
301148,126547,"It was a little uncomfortable at first, but no...",9
301149,126547,I think I'll wear a mask for the rest of my li...,9
301150,126547,I think I should accept it as a habit now.,9


In [5]:
dsm_data_count = dsm_data.groupby('label').count()
dsm_cnt = dsm_data_count[['id']]
dsm_cnt.columns = ['count']
dsm_cnt.T

label,0,1,2,3,4,5,6,7,8,9
count,88602,2089,5252,7290,716,1757,42291,2479,50263,100413


In [6]:
label = dict()

label[0] = '우울'
label[1] = '무기력'
label[2] = '급격한 체중(식욕)변화'
label[3] = '수면장애'
label[4] = '정서불안'
label[5] = '피로'
label[6] = '과도한 죄책감 및 무가치함'
label[7] = '인지기능저하'
label[8] = '자살충동'
label

{0: '우울',
 1: '무기력',
 2: '급격한 체중(식욕)변화',
 3: '수면장애',
 4: '정서불안',
 5: '피로',
 6: '과도한 죄책감 및 무가치함',
 7: '인지기능저하',
 8: '자살충동'}

In [10]:
dsm_data2 = dsm_data.copy()
dsm_data2

,id,text,label
0,al526,"so, am i depressed? i'm honestly unsure myself...",0
1,as8qg,"i originally put it in askreddit, but realized...",0
2,as8qg,how did you do it? am i even depressed at all?...,0
3,ay63c,have been depressed for approximately 20 years...,0
4,aycre,i don't know if i'm depressed or if i'm being ...,0
...,...,...,...
301147,126547,It is said that the incidence of respiratory d...,9
301148,126547,"It was a little uncomfortable at first, but no...",9
301149,126547,I think I'll wear a mask for the rest of my li...,9
301150,126547,I think I should accept it as a habit now.,9


In [11]:
daily_idx = dsm_data2[dsm_data2.label==9].index.tolist()
dsm_data2.drop(daily_idx, inplace=True)
dsm_data2.reset_index(drop=True, inplace=True)
dsm_data2

,id,text,label
0,al526,"so, am i depressed? i'm honestly unsure myself...",0
1,as8qg,"i originally put it in askreddit, but realized...",0
2,as8qg,how did you do it? am i even depressed at all?...,0
3,ay63c,have been depressed for approximately 20 years...,0
4,aycre,i don't know if i'm depressed or if i'm being ...,0
...,...,...,...
200734,t3_43axlc,"but every single god damn day, i have homicida...",8
200735,t3_4e9tie,having suicidal or homicidal thoughts can occu...,8
200736,t3_4fh7w9,i am swinging wildly between suicidal depressi...,8
200737,t3_4gu8xu,i wrote a gory story; i helped the homicidal f...,8


In [15]:
dsm_data_count2 = dsm_data2.groupby('label').count()
dsm_cnt2 = dsm_data_count2[['id']]
dsm_cnt2.columns = ['count']
dsm_cnt2.T

label,0,1,2,3,4,5,6,7,8
count,88602,2089,5252,7290,716,1757,42291,2479,50263


In [12]:
dsm_data2.duplicated('text').sum()

0

In [13]:
dsm_data2.to_csv(os.path.join(data_path, 'dsm_data2.csv'), index=False)

In [36]:
dsm_sample2 = dsm_data2.copy()

In [37]:
dsm_0_idx = dsm_sample2[dsm_sample2.label==0].sample(40000, random_state=42).index.tolist()
dsm_6_idx = dsm_sample2[dsm_sample2.label==6].sample(10000, random_state=42).index.tolist()
dsm_8_idx = dsm_sample2[dsm_sample2.label==8].sample(10000, random_state=42).index.tolist()
dsm_8_idx[:3]

[182160, 154213, 185551]

In [38]:
dsm_sample2 = dsm_sample2[dsm_sample2.label!=0]
dsm_sample2 = dsm_sample2[dsm_sample2.label!=6]
dsm_sample2 = dsm_sample2[dsm_sample2.label!=8]
dsm_sample2_idx = dsm_sample2.index.tolist()
dsm_sample2_idx.extend(dsm_0_idx)
dsm_sample2_idx.extend(dsm_6_idx)
dsm_sample2_idx.extend(dsm_8_idx)
dsm_sample2 = dsm_data2.loc[dsm_sample2_idx]
dsm_sample2.reset_index(inplace=True, drop=True)

In [39]:
dsm_sample2

,id,text,label
0,dww5h,i'm lethargic and not any fun to be around,1
1,eals6,"i spend most of the day incredibly lethargic, ...",1
2,eus4x,"i feel slow and lethargic and foggy, and i've ...",1
3,fpz9z,and i feel too lethargic to try many times,1
4,gqwdt,are you guys lethargic? did it go away once yo...,1
...,...,...,...
79578,4yxtqp,i am 20 years old and have suffered from depre...,8
79579,t3_194c2h,i've even attempted suicide,8
79580,mnbwi,"if you are wondering, i have previously though...",8
79581,t3_302nf2,that is why this statement rings true and why ...,8


In [40]:
dsm_data_count2 = dsm_sample2.groupby('label').count()
dsm_cnt2 = dsm_data_count2[['id']]
dsm_cnt2.columns = ['count']
dsm_cnt2.T

label,0,1,2,3,4,5,6,7,8
count,40000,2089,5252,7290,716,1757,10000,2479,10000


In [41]:
dsm_sample2.to_csv(os.path.join(data_path, 'dsm_sample2.csv'), index=False)